## Passo 1: O Que É Logging?

Logging é **registrar o que acontece** no seu programa.

### Por que usar logging?

#### Sem logging:
```python
# Você não sabe o que aconteceu
resultado = processar_dados()
# Deu erro? Quando? Onde?
```

#### Com logging:
```python
logger.info("Iniciando processamento")
resultado = processar_dados()
logger.info("Processamento concluído")
# Você sabe exatamente o que aconteceu!
```

### Analogia: Diário de Bordo

Imagine um navio:
```
08:00 - Partimos do porto
10:30 - Encontramos tempestade
12:00 - Tempestade passou
15:00 - Chegamos ao destino
```

Logs fazem o mesmo para seu código!

## Passo 2: Níveis de Log

Existem diferentes níveis de importância:

### Os 5 níveis principais:

1. **DEBUG**: Informações detalhadas para diagnóstico
   - Exemplo: "Conectando à URL https://..."

2. **INFO**: Informações gerais
   - Exemplo: "Processando 10 filmes"

3. **WARNING**: Algo inesperado, mas não é erro
   - Exemplo: "Filme sem orçamento informado"

4. **ERROR**: Erro que afeta uma operação
   - Exemplo: "Falha ao buscar filme 123"

5. **CRITICAL**: Erro grave que para o programa
   - Exemplo: "API key inválida"

## Passo 3: Configurar Logging

Vamos criar uma configuração de logging.

In [1]:
import logging
from pathlib import Path
from datetime import datetime

# Criar pasta de logs
logs_path = Path("logs")
logs_path.mkdir(exist_ok=True)

print(f"Pasta de logs criada: {logs_path}")

Pasta de logs criada: logs


In [2]:
# Configurar logging
def setup_logging(name: str = "pipeline") -> logging.Logger:
    """Configura sistema de logging."""
    
    # Criar logger
    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)
    
    # Remover handlers anteriores (para evitar duplicatas)
    logger.handlers.clear()
    
    # Formato das mensagens
    formato = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    formatter = logging.Formatter(formato)
    
    # Handler para arquivo
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_handler = logging.FileHandler(
        f"logs/{name}_{timestamp}.log",
        encoding='utf-8'
    )
    file_handler.setLevel(logging.DEBUG)
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    
    # Handler para console
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)
    
    return logger

print("Função setup_logging criada!")

Função setup_logging criada!


### Testar logging:

In [3]:
# Criar logger
logger = setup_logging("teste")

# Testar diferentes níveis
logger.debug("Mensagem de DEBUG")
logger.info("Mensagem de INFO")
logger.warning("Mensagem de WARNING")
logger.error("Mensagem de ERROR")

print("\nMensagens enviadas para o log!")

2026-01-18 13:38:44,111 - teste - INFO - Mensagem de INFO
2026-01-18 13:38:44,111 - teste - WARNING - Mensagem de WARNING
2026-01-18 13:38:44,111 - teste - ERROR - Mensagem de ERROR



Mensagens enviadas para o log!


Note que DEBUG não apareceu no console (porque configuramos INFO como mínimo), mas está no arquivo!

### Ver arquivo de log:

In [4]:
# Listar arquivos de log
!ls -lh logs/

total 8
-rw-r--r--@ 1 thiago.pinto  staff   242B Jan 18 13:38 teste_20260118_133844.log


In [5]:
# Ver conteúdo do último log
import glob

logs = sorted(glob.glob("logs/*.log"))
if logs:
    ultimo_log = logs[-1]
    print(f"Conteúdo de {ultimo_log}:")
    print()
    with open(ultimo_log) as f:
        print(f.read())

Conteúdo de logs/teste_20260118_133844.log:

2026-01-18 13:38:44,111 - teste - DEBUG - Mensagem de DEBUG
2026-01-18 13:38:44,111 - teste - INFO - Mensagem de INFO
2026-01-18 13:38:44,111 - teste - WARNING - Mensagem de WARNING
2026-01-18 13:38:44,111 - teste - ERROR - Mensagem de ERROR



## Passo 4: Criar Cliente TMDb com Logging

Vamos criar uma classe para interagir com a API.

In [6]:
import requests
import json
from time import sleep
from typing import Optional, List

class TMDbClient:
    """Cliente para API do TMDb com logging."""
    
    def __init__(self, config_path: str = "config.json"):
        """Inicializa cliente."""
        self.logger = logging.getLogger("TMDbClient")
        
        # Carregar configuração
        with open(config_path) as f:
            config = json.load(f)
        
        self.api_key = config['tmdb']['api_key']
        self.base_url = config['tmdb']['base_url']
        self.language = config['tmdb']['language']
        
        self.logger.info("Cliente TMDb inicializado")
    
    def get_movie(self, movie_id: int, max_retries: int = 3) -> Optional[dict]:
        """Busca detalhes de um filme."""
        self.logger.debug(f"Buscando filme {movie_id}")
        
        for tentativa in range(1, max_retries + 1):
            try:
                url = f"{self.base_url}/movie/{movie_id}"
                
                response = requests.get(
                    url,
                    params={
                        'api_key': self.api_key,
                        'language': self.language
                    },
                    timeout=10
                )
                
                if response.status_code == 200:
                    self.logger.info(f"Filme {movie_id} obtido com sucesso")
                    return response.json()
                elif response.status_code == 404:
                    self.logger.warning(f"Filme {movie_id} não encontrado")
                    return None
                elif response.status_code == 429:
                    self.logger.warning("Limite de requisições atingido, aguardando...")
                    sleep(2)
                else:
                    self.logger.error(
                        f"Erro {response.status_code} ao buscar filme {movie_id} "
                        f"(tentativa {tentativa}/{max_retries})"
                    )
            
            except requests.exceptions.Timeout:
                self.logger.error(f"Timeout na tentativa {tentativa}/{max_retries}")
            except Exception as e:
                self.logger.error(f"Erro inesperado: {e}")
            
            if tentativa < max_retries:
                sleep(1)
        
        self.logger.error(f"Falha ao obter filme {movie_id} após {max_retries} tentativas")
        return None
    
    def get_popular(self, page: int = 1) -> List[dict]:
        """Busca filmes populares."""
        self.logger.info(f"Buscando filmes populares (página {page})")
        
        url = f"{self.base_url}/movie/popular"
        
        response = requests.get(
            url,
            params={
                'api_key': self.api_key,
                'language': self.language,
                'page': page
            }
        )
        
        if response.status_code == 200:
            data = response.json()
            filmes = data['results']
            self.logger.info(f"Obtidos {len(filmes)} filmes populares")
            return filmes
        else:
            self.logger.error(f"Erro ao buscar filmes populares: {response.status_code}")
            return []

print("Classe TMDbClient criada!")

Classe TMDbClient criada!


### Testar cliente:

In [7]:
# Criar cliente
client = TMDbClient()

# Testar busca de filme
filme = client.get_movie(550)

if filme:
    print(f"\nFilme: {filme['title']}")


Filme: Clube da Luta


## Passo 5: Criar Pipeline Completo

Agora vamos juntar tudo!

In [8]:
from models import Movie
from time import sleep

class MoviePipeline:
    """Pipeline completo de coleta de filmes."""
    
    def __init__(self, config_path: str = "config.json"):
        """Inicializa pipeline."""
        self.logger = setup_logging("pipeline")
        self.logger.info("Inicializando MoviePipeline")
        
        self.client = TMDbClient(config_path)
        self.dados_path = Path("dados")
        self.dados_path.mkdir(exist_ok=True)
        
        self.filmes_coletados = []
        self.erros = []
        
        self.logger.info("MoviePipeline inicializado")
    
    def coletar_populares(self, num_filmes: int = 10) -> List[Movie]:
        """Coleta filmes populares."""
        self.logger.info(f"Iniciando coleta de {num_filmes} filmes populares")
        
        # Buscar lista de populares
        populares = self.client.get_popular()
        
        if not populares:
            self.logger.error("Não foi possível obter lista de filmes populares")
            return []
        
        # Limitar quantidade
        movie_ids = [f['id'] for f in populares[:num_filmes]]
        self.logger.info(f"Coletando detalhes de {len(movie_ids)} filmes")
        
        # Buscar detalhes
        for i, movie_id in enumerate(movie_ids, 1):
            self.logger.info(f"Progresso: {i}/{len(movie_ids)}")
            
            data = self.client.get_movie(movie_id)
            
            if data:
                try:
                    filme = Movie.from_json(data)
                    self.filmes_coletados.append(filme)
                    self.logger.info(f"Filme processado: {filme.title}")
                except Exception as e:
                    self.logger.error(f"Erro ao processar filme {movie_id}: {e}")
                    self.erros.append((movie_id, str(e)))
            
            # Rate limiting
            sleep(0.5)
        
        self.logger.info(
            f"Coleta concluída: {len(self.filmes_coletados)} sucesso, "
            f"{len(self.erros)} erros"
        )
        
        return self.filmes_coletados
    
    def salvar_filmes(self) -> None:
        """Salva filmes coletados."""
        self.logger.info(f"Salvando {len(self.filmes_coletados)} filmes")
        
        for filme in self.filmes_coletados:
            arquivo = self.dados_path / f"filme_{filme.id}.json"
            
            # Converter para dicionário
            data = {
                'id': filme.id,
                'title': filme.title,
                'original_title': filme.original_title,
                'release_date': filme.release_date,
                'vote_average': filme.vote_average,
                'vote_count': filme.vote_count,
                'overview': filme.overview,
                'popularity': filme.popularity,
                'adult': filme.adult,
                'original_language': filme.original_language,
                'budget': filme.budget,
                'revenue': filme.revenue,
                'runtime': filme.runtime
            }
            
            with open(arquivo, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=2, ensure_ascii=False)
            
            self.logger.debug(f"Filme {filme.id} salvo em {arquivo}")
        
        self.logger.info("Todos os filmes salvos")
    
    def criar_indice(self) -> None:
        """Cria índice de filmes."""
        self.logger.info("Criando índice")
        
        indice = []
        for filme in self.filmes_coletados:
            indice.append({
                'id': filme.id,
                'title': filme.title,
                'release_date': filme.release_date,
                'vote_average': filme.vote_average
            })
        
        indice_path = self.dados_path / "indice_filmes.json"
        with open(indice_path, 'w', encoding='utf-8') as f:
            json.dump(indice, f, indent=2, ensure_ascii=False)
        
        self.logger.info(f"Índice criado: {len(indice)} filmes")
    
    def gerar_relatorio(self) -> None:
        """Gera relatório da coleta."""
        self.logger.info("Gerando relatório")
        
        total = len(self.filmes_coletados)
        
        if total == 0:
            self.logger.warning("Nenhum filme coletado")
            return
        
        # Estatísticas
        notas = [f.vote_average for f in self.filmes_coletados]
        nota_media = sum(notas) / len(notas)
        
        bem_avaliados = len([f for f in self.filmes_coletados if f.is_well_rated(7.0)])
        
        lucrativos = len([f for f in self.filmes_coletados if f.is_profitable()])
        
        # Log do relatório
        self.logger.info("=" * 50)
        self.logger.info("RELATÓRIO DA COLETA")
        self.logger.info("=" * 50)
        self.logger.info(f"Total de filmes: {total}")
        self.logger.info(f"Nota média: {nota_media:.2f}/10")
        self.logger.info(f"Bem avaliados (>= 7.0): {bem_avaliados}")
        self.logger.info(f"Lucrativos: {lucrativos}")
        self.logger.info(f"Erros: {len(self.erros)}")
        self.logger.info("=" * 50)
    
    def executar(self, num_filmes: int = 10) -> None:
        """Executa pipeline completo."""
        self.logger.info("INICIANDO EXECUÇÃO DO PIPELINE")
        self.logger.info("=" * 50)
        
        try:
            # Coletar
            self.coletar_populares(num_filmes)
            
            # Salvar
            self.salvar_filmes()
            
            # Índice
            self.criar_indice()
            
            # Relatório
            self.gerar_relatorio()
            
            self.logger.info("=" * 50)
            self.logger.info("PIPELINE CONCLUÍDO COM SUCESSO")
        
        except Exception as e:
            self.logger.critical(f"ERRO CRÍTICO NO PIPELINE: {e}")
            raise

print("Classe MoviePipeline criada!")

Classe MoviePipeline criada!


## Passo 6: Executar Pipeline

Vamos executar o pipeline completo!

In [9]:
# Criar pipeline
pipeline = MoviePipeline()

print("Pipeline criado!")
print("Preparando para executar...")

2026-01-18 13:38:44,704 - pipeline - INFO - Inicializando MoviePipeline
2026-01-18 13:38:44,705 - pipeline - INFO - MoviePipeline inicializado


Pipeline criado!
Preparando para executar...


In [10]:
# Executar com 10 filmes
pipeline.executar(num_filmes=10)

2026-01-18 13:38:44,709 - pipeline - INFO - INICIANDO EXECUÇÃO DO PIPELINE
2026-01-18 13:38:44,709 - pipeline - INFO - ==================================================
2026-01-18 13:38:44,710 - pipeline - INFO - Iniciando coleta de 10 filmes populares
2026-01-18 13:38:44,986 - pipeline - INFO - Coletando detalhes de 10 filmes
2026-01-18 13:38:44,986 - pipeline - INFO - Progresso: 1/10
2026-01-18 13:38:45,217 - pipeline - INFO - Filme processado: Predador: Terras Selvagens
2026-01-18 13:38:45,720 - pipeline - INFO - Progresso: 2/10
2026-01-18 13:38:45,930 - pipeline - INFO - Filme processado: Icefall
2026-01-18 13:38:46,433 - pipeline - INFO - Progresso: 3/10
2026-01-18 13:38:46,651 - pipeline - INFO - Filme processado: Dinheiro Suspeito
2026-01-18 13:38:47,153 - pipeline - INFO - Progresso: 4/10
2026-01-18 13:38:47,362 - pipeline - INFO - Filme processado: Avatar: Fogo e Cinzas
2026-01-18 13:38:47,866 - pipeline - INFO - Progresso: 5/10
2026-01-18 13:38:48,162 - pipeline - INFO - Fil

## Passo 7: Analisar Logs

Vamos ver o que foi registrado.

In [11]:
# Listar logs
!ls -lht logs/ | head -10

total 16
-rw-r--r--@ 1 thiago.pinto  staff   3.9K Jan 18 13:38 pipeline_20260118_133844.log
-rw-r--r--@ 1 thiago.pinto  staff   242B Jan 18 13:38 teste_20260118_133844.log


In [12]:
# Ver último log do pipeline
logs_pipeline = sorted(glob.glob("logs/pipeline_*.log"))

if logs_pipeline:
    ultimo_log = logs_pipeline[-1]
    print(f"Conteúdo de {ultimo_log}:")
    print()
    
    with open(ultimo_log) as f:
        linhas = f.readlines()
    
    # Mostrar primeiras e últimas linhas
    print("Primeiras 10 linhas:")
    print("-" * 80)
    for linha in linhas[:10]:
        print(linha.strip())
    
    print()
    print("Últimas 10 linhas:")
    print("-" * 80)
    for linha in linhas[-10:]:
        print(linha.strip())

Conteúdo de logs/pipeline_20260118_133844.log:

Primeiras 10 linhas:
--------------------------------------------------------------------------------
2026-01-18 13:38:44,704 - pipeline - INFO - Inicializando MoviePipeline
2026-01-18 13:38:44,705 - pipeline - INFO - MoviePipeline inicializado
2026-01-18 13:38:44,709 - pipeline - INFO - INICIANDO EXECUÇÃO DO PIPELINE
2026-01-18 13:38:44,709 - pipeline - INFO - ==================================================
2026-01-18 13:38:44,710 - pipeline - INFO - Iniciando coleta de 10 filmes populares
2026-01-18 13:38:44,986 - pipeline - INFO - Coletando detalhes de 10 filmes
2026-01-18 13:38:44,986 - pipeline - INFO - Progresso: 1/10
2026-01-18 13:38:45,217 - pipeline - INFO - Filme processado: Predador: Terras Selvagens
2026-01-18 13:38:45,720 - pipeline - INFO - Progresso: 2/10
2026-01-18 13:38:45,930 - pipeline - INFO - Filme processado: Icefall

Últimas 10 linhas:
------------------------------------------------------------------------------

## Passo 8: Analisar Resultados

Vamos ver os filmes coletados.

In [13]:
# Carregar índice
with open("dados/indice_filmes.json") as f:
    indice = json.load(f)

print(f"Total de filmes no índice: {len(indice)}")
print()
print("Filmes coletados:")
print()

for i, filme in enumerate(indice, 1):
    print(f"{i:2}. {filme['title']:40} {filme['release_date'][:4]}  {filme['vote_average']}/10")

Total de filmes no índice: 10

Filmes coletados:

 1. Predador: Terras Selvagens               2025  7.794/10
 2. Icefall                                  2025  6.677/10
 3. Dinheiro Suspeito                        2026  7.182/10
 4. Avatar: Fogo e Cinzas                    2025  7.356/10
 5. Zootopia 2                               2025  7.61/10
 6. O Tanque de Guerra                       2025  7.021/10
 7. A Empregada                              2025  7.2/10
 8. Bingo Bongo                              1982  5.711/10
 9. A Guerra dos Mundos                      2025  4.222/10
10. Boca de Fumo                             2025  6.5/10


## Passo 9: Estatísticas Finais

In [14]:
# Carregar filmes como objetos
filmes = pipeline.filmes_coletados

print("Estatísticas da Coleta")
print("=" * 50)
print(f"Total de filmes: {len(filmes)}")
print()

# Notas
notas = [f.vote_average for f in filmes]
print(f"Nota média: {sum(notas) / len(notas):.2f}/10")
print(f"Melhor nota: {max(notas)}/10")
print(f"Pior nota: {min(notas)}/10")
print()

# Bem avaliados
bem_avaliados = [f for f in filmes if f.is_well_rated(7.0)]
print(f"Filmes bem avaliados (>= 7.0): {len(bem_avaliados)}")
print()

# Lucrativos
lucrativos = [f for f in filmes if f.is_profitable()]
print(f"Filmes lucrativos: {len(lucrativos)}")

if lucrativos:
    lucros = [f.get_profit() for f in lucrativos if f.get_profit()]
    if lucros:
        print(f"Lucro total: ${sum(lucros):,}")
        print(f"Lucro médio: ${sum(lucros) // len(lucros):,}")

Estatísticas da Coleta
Total de filmes: 10

Nota média: 6.73/10
Melhor nota: 7.794/10
Pior nota: 4.222/10

Filmes bem avaliados (>= 7.0): 6

Filmes lucrativos: 4
Lucro total: $2,592,385,267
Lucro médio: $648,096,316


## Passo 10: Salvar Pipeline em Arquivo

Vamos salvar nosso código em arquivos Python.

In [15]:
# Arquivo: tmdb_client.py
codigo_client = '''"""Cliente para API do TMDb."""

import logging
import requests
import json
from time import sleep
from typing import Optional, List


class TMDbClient:
    """Cliente para API do TMDb com logging."""
    
    def __init__(self, config_path: str = "config.json"):
        """Inicializa cliente."""
        self.logger = logging.getLogger("TMDbClient")
        
        with open(config_path) as f:
            config = json.load(f)
        
        self.api_key = config['tmdb']['api_key']
        self.base_url = config['tmdb']['base_url']
        self.language = config['tmdb']['language']
        
        self.logger.info("Cliente TMDb inicializado")
    
    def get_movie(self, movie_id: int, max_retries: int = 3) -> Optional[dict]:
        """Busca detalhes de um filme."""
        self.logger.debug(f"Buscando filme {movie_id}")
        
        for tentativa in range(1, max_retries + 1):
            try:
                url = f"{self.base_url}/movie/{movie_id}"
                
                response = requests.get(
                    url,
                    params={
                        'api_key': self.api_key,
                        'language': self.language
                    },
                    timeout=10
                )
                
                if response.status_code == 200:
                    self.logger.info(f"Filme {movie_id} obtido com sucesso")
                    return response.json()
                elif response.status_code == 404:
                    self.logger.warning(f"Filme {movie_id} não encontrado")
                    return None
                elif response.status_code == 429:
                    self.logger.warning("Limite de requisições atingido")
                    sleep(2)
                else:
                    self.logger.error(f"Erro {response.status_code}")
            
            except requests.exceptions.Timeout:
                self.logger.error(f"Timeout na tentativa {tentativa}")
            except Exception as e:
                self.logger.error(f"Erro: {e}")
            
            if tentativa < max_retries:
                sleep(1)
        
        return None
    
    def get_popular(self, page: int = 1) -> List[dict]:
        """Busca filmes populares."""
        self.logger.info(f"Buscando filmes populares (página {page})")
        
        url = f"{self.base_url}/movie/popular"
        
        response = requests.get(
            url,
            params={
                'api_key': self.api_key,
                'language': self.language,
                'page': page
            }
        )
        
        if response.status_code == 200:
            data = response.json()
            self.logger.info(f"Obtidos {len(data['results'])} filmes")
            return data['results']
        else:
            self.logger.error(f"Erro: {response.status_code}")
            return []
'''

with open("tmdb_client.py", "w") as f:
    f.write(codigo_client)

print("tmdb_client.py criado!")

tmdb_client.py criado!


## Passo 11: Commitar Tudo

Vamos versionar nosso trabalho final!

In [16]:
!git status

On branch main
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	dados/
	logs/
	tmdb_client.py

nothing added to commit but untracked files present (use "git add" to track)


In [17]:
!git add .
!git status

On branch main
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   dados/indice_filmes.json
	new file:   logs/pipeline_20260118_133844.log
	new file:   logs/teste_20260118_133844.log
	new file:   tmdb_client.py



In [18]:
!git commit -m "feat: adiciona pipeline completo com logging e execução automatizada"

[main e52067d] feat: adiciona pipeline completo com logging e execução automatizada
 4 files changed, 205 insertions(+)
 create mode 100644 dados/indice_filmes.json
 create mode 100644 logs/pipeline_20260118_133844.log
 create mode 100644 logs/teste_20260118_133844.log
 create mode 100644 tmdb_client.py


In [19]:
# Ver histórico
!git log --oneline

e52067d (HEAD -> main) feat: adiciona pipeline completo com logging e execução automatizada
0b02258 feat: adiciona requirements.txt com dependências
3ad313b (feature/logging) docs: adiciona seção de contribuição ao README
ea5fc55 docs: adiciona README com instruções de uso
49ef702 feat: cria pipeline inicial de coleta de filmes


## Resumo do Bloco 5

### O que fizemos:

1. Aprendemos o que é logging
2. Configuramos sistema de logs (arquivo + console)
3. Criamos cliente TMDb com logging
4. Criamos pipeline completo integrado
5. Executamos coleta automatizada
6. Geramos relatórios
7. Analisamos resultados
8. Versionamos tudo com Git

### O que aprendemos:

- Por que logging é importante
- Níveis de log (DEBUG, INFO, WARNING, ERROR, CRITICAL)
- Como configurar logging em Python
- Como integrar componentes em um pipeline
- Como automatizar execução
- Como gerar relatórios
- Importância de boas práticas de código

### Estrutura Final:

```
.
├── .git/                    # Repositório Git
├── .gitignore               # Arquivos ignorados
├── config.json              # Configuração (não versionado)
├── config.json.example      # Exemplo de configuração
├── models.py                # Classes de dados
├── tmdb_client.py           # Cliente API
├── README.md                # Documentação
├── requirements.txt         # Dependências
├── 01_setup.ipynb           # Setup inicial
├── 02_coleta_dados.ipynb    # Coleta de dados
├── 03_estruturacao.ipynb    # Estruturação
├── 04_versionamento.ipynb   # Versionamento
├── 05_pipeline_completo.ipynb # Pipeline completo
├── dados/                   # Dados coletados
│   ├── filme_*.json
│   └── indice_filmes.json
└── logs/                    # Logs de execução
    └── pipeline_*.log
```

## Verificação Final Completa

In [20]:
print("VERIFICAÇÃO FINAL - AULA 1 COMPLETA")
print("=" * 60)

import os

# 1. Git
print("\n1. VERSIONAMENTO")
print("-" * 60)
if os.path.exists(".git"):
    print("✓ Repositório Git inicializado")
    !git log --oneline | wc -l
else:
    print("✗ Git não inicializado")

# 2. Arquivos Python
print("\n2. CÓDIGO PYTHON")
print("-" * 60)
arquivos_python = ['models.py', 'tmdb_client.py']
for arquivo in arquivos_python:
    existe = os.path.exists(arquivo)
    print(f"{'✓' if existe else '✗'} {arquivo}")

# 3. Notebooks
print("\n3. NOTEBOOKS")
print("-" * 60)
notebooks = [
    '01_setup.ipynb',
    '02_coleta_dados.ipynb',
    '03_estruturacao.ipynb',
    '04_versionamento.ipynb',
    '05_pipeline_completo.ipynb'
]
for nb in notebooks:
    existe = os.path.exists(nb)
    print(f"{'✓' if existe else '✗'} {nb}")

# 4. Documentação
print("\n4. DOCUMENTAÇÃO")
print("-" * 60)
docs = ['.gitignore', 'README.md', 'requirements.txt', 'config.json.example']
for doc in docs:
    existe = os.path.exists(doc)
    print(f"{'✓' if existe else '✗'} {doc}")

# 5. Dados
print("\n5. DADOS COLETADOS")
print("-" * 60)
if os.path.exists("dados"):
    filmes = len(list(Path("dados").glob("filme_*.json")))
    print(f"✓ Pasta dados existe")
    print(f"  {filmes} filmes coletados")
    
    if os.path.exists("dados/indice_filmes.json"):
        print("✓ Índice criado")
else:
    print("✗ Pasta dados não existe")

# 6. Logs
print("\n6. LOGS")
print("-" * 60)
if os.path.exists("logs"):
    logs = len(list(Path("logs").glob("*.log")))
    print(f"✓ Pasta logs existe")
    print(f"  {logs} arquivos de log")
else:
    print("✗ Pasta logs não existe")

print("\n" + "=" * 60)
print("AULA 1: Como Nasce um Pipeline de Dados Versionado?")
print("STATUS: COMPLETA ✓")
print("=" * 60)

VERIFICAÇÃO FINAL - AULA 1 COMPLETA

1. VERSIONAMENTO
------------------------------------------------------------
✓ Repositório Git inicializado
       5

2. CÓDIGO PYTHON
------------------------------------------------------------
✓ models.py
✓ tmdb_client.py

3. NOTEBOOKS
------------------------------------------------------------
✓ 01_setup.ipynb
✓ 02_coleta_dados.ipynb
✓ 03_estruturacao.ipynb
✓ 04_versionamento.ipynb
✓ 05_pipeline_completo.ipynb

4. DOCUMENTAÇÃO
------------------------------------------------------------
✓ .gitignore
✓ README.md
✓ requirements.txt
✓ config.json.example

5. DADOS COLETADOS
------------------------------------------------------------
✓ Pasta dados existe
  11 filmes coletados
✓ Índice criado

6. LOGS
------------------------------------------------------------
✓ Pasta logs existe
  2 arquivos de log

AULA 1: Como Nasce um Pipeline de Dados Versionado?
STATUS: COMPLETA ✓


## 🎉 Parabéns!

Você completou a **Aula 1: Como Nasce um Pipeline de Dados Versionado?**

### O que você aprendeu:

1. **Dados Reais** - Coletar de APIs com requests
2. **Python** - Classes, dataclasses, validações
3. **Git** - Versionar código, commits, branches
4. **Logging** - Registrar execução
5. **Pipeline** - Integrar tudo em fluxo automatizado

### Habilidades desenvolvidas:

- ✅ Fazer requisições HTTP
- ✅ Estruturar dados com classes
- ✅ Versionar código com Git
- ✅ Implementar logging
- ✅ Criar pipelines automatizados
- ✅ Tratar erros
- ✅ Documentar código

### Próximos Passos:

Na **Aula 2**, você vai aprender:
- Como guardar dados em banco SQL
- Como criar API REST
- Como transformar dados em sistema acessível
